In [ ]:
import pandas as pd, numpy as np

import GEOparse, pyhgnc, pickle, copy

In [4]:
def get_gene_fc_dict(gene_fc_dataframe_, sample_name, duplicate_handler=np.mean, hgnc=True, hgnc_path="", query=None):
    gene_fc_dataframe = copy.deepcopy(gene_fc_dataframe_)[[sample_name]]
    gene_fc_dict = dict()
    geo_hgnc_ = False
    try:
        geo_hgnc_dict = pickle.load(open("datasets/geo_to_hgnc_ids","rb"))
    except FileNotFoundError:
        geo_hgnc_ = True
        geo_hgnc_dict = dict()
    for row in gene_fc_dataframe.iterrows():
        gene_symbol = row[0]
        if hgnc:
            try:
                if geo_hgnc_:
                    geo_hgnc_dict[gene_symbol] = "HGNC:{}".format(query.hgnc(symbol=gene_symbol)[0].identifier)
                gene_symbol = geo_hgnc_dict[gene_symbol]
            except (KeyError, IndexError) as e:
                continue
        gene_fc = row[1]
        gene_fc_dict.setdefault(gene_symbol, [])
        gene_fc_dict[gene_symbol].append(gene_fc)
    for gene in gene_fc_dict:
        gene_fc_dict[gene] = duplicate_handler(gene_fc_dict[gene])
    if geo_hgnc_ and hgnc_path:
        with open("datasets/geo_to_hgnc_ids","wb") as f:
            pickle.dump(geo_hgnc_dict, f)
    print ("Dict for {} addedd successfully with {} samples".format(sample_name, len(gene_fc_dict)))
    return gene_fc_dict

In [8]:
def parse_database(geo_database_name, index_column="IDENTIFIER", hgnc_path="", labels={}):
    geo_df = GEOparse.get_GEO(geo=geo_database_name)\
                                .table\
                                .set_index(index_column).dropna()
    geo_df = geo_df[[col for col in geo_df.columns if "GSM" in col]]
    query = pyhgnc.query()
    X = list()
    y = list()
    for sample in geo_df.columns:
        if sample in labels:
            X.append(get_gene_fc_dict(geo_df, sample, query=query, hgnc_path=hgnc_path))
            y.append(labels[sample])
    return X,y
    

In [9]:
labels = {}
hs = "GSM1599181,GSM1599182,GSM1599183".split(',')
bcs = "GSM1599177,GSM1599178,GSM1599179,GSM1599180".split(',')
for s in hs:
    labels[s] = "healthy"
for b in bcs:
    labels[b] = "bc"

In [10]:
gene_data, labels = parse_database("GDS5819",hgnc_path="datasets/geo_to_hgnc_ids", labels=labels)

27-Dec-2017 17:29:36 INFO GEOparse - File already exist: using local version.
27-Dec-2017 17:29:36 INFO GEOparse - Parsing ./GDS5819.soft.gz: 
27-Dec-2017 17:29:36 DEBUG GEOparse - DATABASE: Geo
27-Dec-2017 17:29:36 DEBUG GEOparse - DATASET: GDS5819
27-Dec-2017 17:29:36 DEBUG GEOparse - SUBSET: GDS5819_1
27-Dec-2017 17:29:36 DEBUG GEOparse - SUBSET: GDS5819_2
27-Dec-2017 17:29:36 DEBUG GEOparse - SUBSET: GDS5819_3
27-Dec-2017 17:29:36 DEBUG GEOparse - SUBSET: GDS5819_4
27-Dec-2017 17:29:36 DEBUG GEOparse - DATASET: GDS5819
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  return DataFrame.from_csv(StringIO(data), index_col=None, sep="\t")


Dict for GSM1599177 addedd successfully with 9745 samples
Dict for GSM1599178 addedd successfully with 9745 samples
Dict for GSM1599179 addedd successfully with 9745 samples
Dict for GSM1599180 addedd successfully with 9745 samples
Dict for GSM1599181 addedd successfully with 9745 samples
Dict for GSM1599182 addedd successfully with 9745 samples
Dict for GSM1599183 addedd successfully with 9745 samples


In [11]:
pickle.dump(gene_data, open("datasets/gene_data_X","wb"))
pickle.dump(labels, open("datasets/gene_data_y","wb"))